<a href="https://colab.research.google.com/github/leandrojgama/Preventivas-Analise-de-Dados/blob/main/Preventivas_Programa%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xlsxwriter
!pip install -U xlrd

     |████████████████████████████████| 149 kB 5.2 MB/s 
     |████████████████████████████████| 96 kB 2.9 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
from google.colab import data_table
import numpy as np
import pandas as pd
from datetime import date,datetime
from dateutil.relativedelta import relativedelta
import xlsxwriter
from google.colab import files
import os

#Carregar Tabelas

##Tratando OS

In [ ]:

os.chdir('/content/drive/MyDrive/Colab Notebooks/FUNED AUTOMAÇÃO/Plano Input') 
print('Carregar arquivo com atividades em execução, pendente de aprovação e planejamento:',
      'o arquivo deve ser .xlsx',sep='\n')
uploaded = files.upload()
osname='OS'+datetime.now().strftime('_%d%m%y')+'.xls'
os.rename(tuple(uploaded.keys())[0],osname)
print(tuple(uploaded.keys())[0],'-rename->',osname)
os_mp=pd.read_excel(osname)
os_mp.head(1)
#Trantando colunas
col=list(os_mp.columns)
l0=list(os_mp.iloc[0])
#print(col,l0,sep='\n')
copy=['SA', 'TA', 'P']
rewrite=['Identificador','Nome']
for i in range(len(col)):
  #--
  if col[i] in copy:
    l0[i]=col[i] 
  if (l0[i] in rewrite) and (col[i][:5]=='Ativi'):    
    l0[i]='Atividade_'+l0[i]
  if (l0[i] in rewrite) and (col[i][:5]=='Ativo'):    
    l0[i]='Ativo_'+l0[i]
  #--

os_mp.columns=l0
os_mp=os_mp.drop(0) 
  




Carregar arquivo com atividades em execução, pendente de aprovação e planejamento:
o arquivo deve ser .xlsx


Saving Listagem_ Atividade - 7b7bcd87.xlsx to Listagem_ Atividade - 7b7bcd87.xlsx
Listagem_ Atividade - 7b7bcd87.xlsx -rename-> OS_290422.xls


##Tratando Plano

In [ ]:

os.chdir('/content/drive/MyDrive/Colab Notebooks/FUNED AUTOMAÇÃO/Plano Input') 
print('Carregar arquivo com plano de manutenção preventiva:',
      'O arquivo deve ser .xlsx',sep='\n')
uploaded = files.upload()
planoname='Plano'+datetime.now().strftime('_%d%m%y')+'.xls'
os.rename(tuple(uploaded.keys())[0],planoname)
print(tuple(uploaded.keys())[0],'-rename->',planoname)
plano=pd.read_excel(planoname)
plano.head(1)
#Trantando colunas
col=list(plano.columns)
l0=list(plano.iloc[0])
#print(col,l0,sep='\n')
copy=['SA', 'TA', 'P']
rewrite=['Identificador','Nome']
for i in range(len(col)):
  #--
  if col[i] in copy:
    l0[i]=col[i] 
  if (l0[i] in rewrite) and (col[i][:5]=='Ativi'):    
    l0[i]='Atividade_'+l0[i]
  if (l0[i] in rewrite) and (col[i][:5]=='Plano'):    
    l0[i]='Plano_'+l0[i]
  #--

plano.columns=l0
plano=plano.drop(0) 
plano=plano.drop(['Atividade_Nome'],axis=1)  
  




Carregar arquivo com plano de manutenção preventiva:
O arquivo deve ser .xlsx


Saving Listagem_ Plano de manutenção preventiva - 7b29821f.xlsx to Listagem_ Plano de manutenção preventiva - 7b29821f.xlsx
Listagem_ Plano de manutenção preventiva - 7b29821f.xlsx -rename-> Plano_290422.xls


In [ ]:

#Arquivo atividade
#os_mp=pd.read_excel('/content/drive/MyDrive/Colab Notebooks/FUNED AUTOMAÇÃO/OS 29-03-2022.xlsx')
os=os_mp[(os_mp.TA=='Atividade programada')&(os_mp.SA=='Execução')].copy()
retirar=pd.read_excel('/content/drive/MyDrive/Colab Notebooks/FUNED AUTOMAÇÃO/Não_checar.xlsx')
retirar=retirar[retirar['Status']=='n']
mp=os_mp[(os_mp.TA=='Manutenção preventiva')].copy()
#Arquivo plano
#plano = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/FUNED AUTOMAÇÃO/Plano de manutenção preventiva 29-03-22.xlsx')
#plano.Atividade_Identificador=[x.split('-')[-1] for x in plano.Atividade_Identificador]#.str.split('-')[-1]
plano["Status"] ="--"
plano["OS_Status"] ="--"
#plano["OS_Descrição"] ="--"
plano["data_status"]="--"
plano["MP_Status"]="--"
ind=plano[(plano["Plano_Identificador"].str.contains("MP-SU-")) | (plano["Plano_Nome"].str.contains("RELSP")  | 
                                                             plano['Plano_Identificador'].isin(retirar['Identificador']))].index
plano.drop(ind,inplace=True)

plano['Data da próxima execução']=pd.to_datetime(plano['Data da próxima execução'],format="%d/%m/%Y")
plano['Data da última execução']=pd.to_datetime(plano['Data da última execução'],format="%d/%m/%Y")
os=os.reset_index(drop=True)
mp=mp.reset_index(drop=True)
plano=plano.reset_index(drop=True)


#Funções

In [ ]:
def get_id_os(ativo):
  ls_os=""
  if ativo in os['Ativo_Identificador'].values:
    for ord in os['Atividade_Identificador'].loc[(os['Ativo_Identificador']==ativo)].values:
      ls_os+=","+ord    
    ls_os=ls_os[1:]
    return ls_os
  else:
    return "--"
def mp_checa_frequencia(data_da_ultima,data_da_proxima,frequencia):
    periodo={"Mês(es)":1,"Ano(s)":12,"Dia(s)":(1/30)}
    du=data_da_ultima
    dp=data_da_proxima
    
    fr=frequencia.split() 
    #print(fr)
    #if fr[1]=='Dia(s)':
    fr=int(fr[0])*float(periodo[fr[1]])  
    umMes=relativedelta(months=+1)  
    #Gerar data Referencia
    if date.today().day>=18: 
      if  date.today().month==12:
          dtToday=date((date.today().year+1),1,1) 
      else:
        m=date.today().month+1
        dtToday=date(date.today().year,m,1)    
    else:
     
      dtToday=date(date.today().year,date.today().month,1)  

    #dtToday=date(2021,12,1)
    #print(type(du),'-',type(fr),'-',type(umMes))
    if pd.isnull(du)==False: 
      intervalo=du+fr*umMes

      if (dtToday<=intervalo):   
           
        if (intervalo.month==dp.month) and (intervalo.year==dp.year):
          return "Em dia"  
          # atualizado       
        elif (intervalo.year==dp.year):
          if (intervalo.month>dp.month):           
            return "Em dia - Adiantado"
          elif (intervalo.month<dp.month): 
            return "Adiado"
        elif (intervalo.year>dp.year):
            return "Adiado"
          
               
      #----------------------------------------    
      else:
        return "Cronograma travado"
    else:
      return 'Não existem preventivas'
      
def mp_checa_pendencia_aprovacao(ativo): 
  ls_mp=""
  ls_mp_pen=''
  ls_mp_prog=''
  ls_mp_exe=''
  ls_mp_rep=''
  dup=''
  duplicado=mp[mp.Ativo_Identificador.duplicated()]
  if ativo in duplicado.values:
    dup='**Duplicado** '


  if ativo in mp['Ativo_Identificador'].values:
    pendente=mp[(mp['SA']=='Aprovação do Relatório') ]
    programacao=mp[(mp['SA']=='Planejamento') ]
    execucao=mp[(mp['SA']=='Execução') ]
    reprovada=mp[(mp['SA']=='Atividade reprovada') ]
    if ativo in pendente['Ativo_Identificador'].values:

      for ord in pendente['Atividade_Identificador'].loc[(pendente['Ativo_Identificador']==ativo)].values: 
              
        ls_mp_pen+=","+str(ord)   
      ls_mp=dup+"Pendente de aprovação: "+ls_mp_pen[1:]
    if ativo in programacao['Ativo_Identificador'].values:
      
      for ord in programacao['Atividade_Identificador'].loc[(programacao['Ativo_Identificador']==ativo)].values:   
                     
        ls_mp_prog+=","+str(ord)   

      ls_mp=dup+"Planejamento: "+ls_mp_prog[1:]
    if ativo in reprovada['Ativo_Identificador'].values:
      
      for ord in reprovada['Atividade_Identificador'].loc[(reprovada['Ativo_Identificador']==ativo)].values:   
                     
        ls_mp_rep+=","+str(ord)   

      ls_mp=dup+"Atividade reprovada: "+ls_mp_prog[1:]
    if ativo in execucao['Ativo_Identificador'].values:
      
      for ord in execucao['Atividade_Identificador'].loc[(execucao['Ativo_Identificador']==ativo)].values:   
                     
        ls_mp_exe+=","+str(ord)   

      ls_mp=dup+"Execução: "+ls_mp_exe[1:]
    return ls_mp
  else:
    return dup+"--"



def mp_checa_corretiva():
  #periodo={"Mês(es)":1,"Ano(s)":12}
  df1=plano.copy()
  
  for linha in range(0, len(df1)):        
    ativo= df1['Plano_Identificador'].loc[linha]  
    du=df1['Data da última execução'].loc[linha].date()
    dp=df1['Data da próxima execução'].loc[linha].date()    
    fr=df1['Frequência'].loc[linha]
    
    checkf=mp_checa_frequencia(du,dp,fr)
    if len(ativo)==15:
      if (ativo[6:] in os['Ativo_Identificador'].values) and (ativo[0:2]=='DI'):
        df1['Status'].loc[linha]='Em corretiva'
        df1['OS_Status'].loc[linha]=get_id_os(ativo[6:])
      df1['MP_Status'].loc[linha]=mp_checa_pendencia_aprovacao(ativo[6:])
      df1["data_status"].loc[linha]=checkf
      df1['Atividade_Identificador'].loc[linha]=ativo[6:9]
    elif len(ativo)==12:          
      if (ativo[3:] in os['Ativo_Identificador'].values) and (ativo[0:2]=='MP'):
        df1['Status'].loc[linha]='Em corretiva'
        df1['OS_Status'].loc[linha]=get_id_os(ativo[3:])
      df1['MP_Status'].loc[linha]=mp_checa_pendencia_aprovacao(ativo[3:])
      df1["data_status"].loc[linha]=checkf
      df1['Atividade_Identificador'].loc[linha]=ativo[3:6] 
    else:
      print(len(ativo),'-',ativo+"Fora do padrão")  
    df1.sort_values(by=['Data da próxima execução'],inplace=True)    

    
  return df1
pl=mp_checa_corretiva()

pl.head()

#pl[pl.Status=="Em corretiva"].count()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,P,Plano_Identificador,Data da próxima execução,Data da última execução,Frequência,Atividade_Identificador,Plano_Nome,Status,OS_Status,data_status,MP_Status
0,Atrasado,MP-CACDB-01A,2020-06-01,2019-10-30,6 Mês(es),CAC,Manutenção Preventiva - Câmara Climática Binde...,Em corretiva,"OS-OUT-011106,OS-MNT-020328",Cronograma travado,Atividade reprovada:
1,Atrasado,MP-LIOAC-04C,2020-06-11,2019-12-11,6 Mês(es),LIO,Manutenção Preventiva - Liofilizador Edwards P...,Em corretiva,"OS-MNT-013085,OS-MNT-012872,OS-MNT-017325,OS-M...",Cronograma travado,Atividade reprovada:
2,Atrasado,MP-CSB-00005,2021-01-01,2018-10-15,6 Mês(es),CSB,Manutenção Preventiva - Cabine de Segurança Bi...,--,--,Cronograma travado,Execução: MP-Trescal-000299
3,Atrasado,MP-IBCBF-02B,2021-03-01,2018-07-10,1 Ano(s),IBC,Manutenção Preventiva - Identificador Bacterio...,Em corretiva,"OS-MNT-011985,OS-MNT-011177,OS-MNT-015726,OS-M...",Cronograma travado,--
4,Atrasado,MP-AUTFS-01A,2021-04-21,2021-01-21,3 Mês(es),AUT,Manutenção Preventiva - Autoclave - SPSF,Em corretiva,"OS-MNT-018470,OS-MNT-020157",Cronograma travado,Atividade reprovada:


In [ ]:
"""return "Em dia"  
          # atualizado       
        elif (intervalo.year==dp.year):
          if (intervalo.month>dp.month):           
            return "Adiado"
          elif (intervalo.month<dp.month): 
            return Em dia - Adiantado """
pl[(pl['data_status']=="Em dia")]
#pl.isnull()

,P,Plano_Identificador,Data da próxima execução,Data da última execução,Frequência,Atividade_Identificador,Plano_Nome,Status,OS_Status,data_status,MP_Status
145,Em dia,MP-CSB-00087,2022-05-03,2021-11-03,6 Mês(es),CSB,FS- Manutenção Preventiva - Cabine de Seguranç...,--,--,Em dia,Planejamento: MP-Trescal-000393
146,Em dia,DI-MP-CTE-00001,2022-05-04,2022-02-04,3 Mês(es),CTE,CD- Manutenção Preventiva - Cintadeira Marche...,--,--,Em dia,Planejamento: MP-DI-SMC-002449
147,Em dia,DI-MP-EMB-00001,2022-05-04,2022-02-04,3 Mês(es),EMB,CD- Manutenção Preventiva - Emblistadeira Camp...,--,--,Em dia,Planejamento: MP-DI-SMC-002451
148,Em dia,DI-MP-SEL-00006,2022-05-04,2022-02-04,3 Mês(es),SEL,CD- Manutenção Preventiva - Seladora Fabrima U...,--,--,Em dia,Planejamento: MP-DI-SMC-002466
149,Em dia,MP-CAP-00014,2022-05-06,2021-05-06,1 Ano(s),CAP,FA- Manutenção Preventiva - Capela de Exaustão...,--,--,Em dia,Planejamento: MP-Trescal-000394
...,...,...,...,...,...,...,...,...,...,...,...
1110,Em dia,MP-ESI-00023,2023-04-25,2022-04-25,1 Ano(s),ESI,LC- Manutenção Preventiva - Estufa de Incubação,--,--,Em dia,--
1111,Em dia,MP-ESI-00022,2023-04-25,2022-04-25,1 Ano(s),ESI,LC- Manutenção Preventiva - Estufa de Incubaçã...,--,--,Em dia,--
1112,Em dia,MP-MIC-00086,2023-04-25,2022-04-25,1 Ano(s),MIC,MS- Manutenção Preventiva - Microscópio Biológ...,--,--,Em dia,--
1113,Em dia,MP-HOM-00002,2023-04-26,2022-04-26,1 Ano(s),HOM,SM- Manutenção Preventiva - Homogeneizador de ...,--,--,Em dia,--


#Exportar e Filtros

In [ ]:
# Criando os dataframes Pandas para armazenar os dados 
df1 = pl.sort_values(by=['Data da próxima execução'])
df2 = pl[(pl.data_status=='Cronograma travado')&(pl.Status=='--')].sort_values(by=['Data da próxima execução'])

df3 = pl[(pl.data_status=='Cronograma travado')&(pl.Status=='Em corretiva')].sort_values(by=['Data da próxima execução'])

# Usando o ExcelWriter, cria um doc .xlsx, usando engine='xlsxwriter'
writer = pd.ExcelWriter('/content/drive/MyDrive/Colab Notebooks/FUNED AUTOMAÇÃO/Plano Output/plano'+
                        datetime.now().strftime('_%d-%m-%y')+'.xlsx', engine='xlsxwriter')

# Armazena cada df em uma planilha diferente do mesmo arquivo

df1.to_excel(writer, sheet_name='Geral',index=False)
df2.to_excel(writer, sheet_name='Programar-Sem Pendencias',index=False)
df3.to_excel(writer, sheet_name='Em corretiva',index=False)

# Fecha o ExcelWriter e gera o arquivo .xlsx
writer.save()

In [ ]:
pl.to_excel('/content/drive/MyDrive/Colab Notebooks/FUNED AUTOMAÇÃO/mp_status.xlsx',index=False)


In [ ]:
mp_status=pl.loc[pl['MP_Status'].str.contains('Pendente')]
mp_status.to_excel('/content/drive/MyDrive/Colab Notebooks/FUNED AUTOMAÇÃO/mp_status.xlsx',index=False)  
#mp_status.to_excel(r'Path to store the exported excel file\File Name.xlsx', index = False)

In [ ]:
pendente=mp[(mp['SA']=='Aprovação do Relatório') ]
programado=mp[(mp['SA']=='Planejamento') ]
execucao=mp[(mp.SA != 'Planejamento') & (mp.SA != 'Aprovação do Relatório')]
pendente
print(mp.shape,
      '-',programado.shape,'-'
      ,pendente.shape,'-'
      ,programacao.shape)

(794, 10) - (384, 10) - (94, 10) - (316, 10)


#Teste Filtros


In [ ]:
equip=['CAP/CAB','CSB','CPE','CAP','FXL']
x=pl[(pl.Atividade_Identificador.isin(equip)) & (pl.data_status!='Em dia') ]
x.to_excel('/content/drive/MyDrive/Colab Notebooks/FUNED AUTOMAÇÃO/MP-Pendencias Trescal.xlsx',index=False)  

In [ ]:
ind=pl[(pl["Identificador"].str.contains("PS-")) & (pl["Status"]=="Em corretiva")]
ind
#plano.info()

,P,Identificador,Data da próxima execução,Data da última execução,Frequência,Nome,Status,OS_Status,data_status
1,Atrasado,DI-MP-APFPS-03A,2021-07-01,2021-06-09,3 Mês(es),Manutenção Preventiva - Ultrafiltro Millipore ...,Em corretiva,OS-OUT-011682,Cronograma travado
4,Atrasado,DI-MP-AUTPS-02A,2021-07-14,2021-05-14,2 Mês(es),Manutenção Preventiva - Autoclave Cisa,Em corretiva,"OS-MNT-015850,OS-MNT-016883",Cronograma travado
43,Em dia,DI-MP-OSRPS-01A,2021-08-09,2021-06-09,2 Mês(es),Manutenção Preventiva - Osmose Reversa Baumer,Em corretiva,OS-MNT-017757,Em dia
50,Em dia,DI-MP-SPAPS-02A,2021-08-01,2021-02-22,6 Mês(es),Manutenção Preventiva - Sistema de Purificação...,Em corretiva,"OS-MNT-013101,OS-MNT-013103,OS-MNT-014007,OS-M...",Em dia
65,Em dia,DI-MP-TAQPS-06A,2021-09-09,2021-06-09,3 Mês(es),Manutenção Preventiva - Tanque Móvel A Imuno,Em corretiva,OS-MNT-017849,Em dia


In [ ]:
mes=pl.loc[(pl["Data da próxima execução"]>='2021-12-01') & (pl["Data da próxima execução"]<='2021-12-31')] #& ((pl.data_status=='Fora da frequencia') | (pl.data_status=='Cronograma travado'))]
#mes.to_excel('/content/drive/MyDrive/Colab Notebooks/FUNED AUTOMAÇÃO/Preventivas hoje.xlsx',index=False)
#help(plano["Data da próxima execução"])
mes.sort_values(by=['Data da próxima execução']).to_excel('/content/drive/MyDrive/Colab Notebooks/FUNED AUTOMAÇÃO/Preventivas 01-12-21_12-21.xlsx',index=False)

In [ ]:
#Geral SMC
pl.sort_values(by=['Data da próxima execução']).to_excel('/content/drive/MyDrive/Colab Notebooks/FUNED AUTOMAÇÃO/Preventivas Geral SMC.xlsx',index=False)

#Teste

In [ ]:
def mp_checa_frequencia(data_da_ultima,data_da_proxima,frequencia):
    periodo={"Mês(es)":1,"Ano(s)":12}
    du=data_da_ultima
    dp=data_da_proxima
    
    fr=frequencia.split() 
    fr=int(fr[0])*int(periodo[fr[1]])  
    umMes=relativedelta(months=+1)  
    #Gerar data Referencia
    if date.today().day>=18:
      m=date.today().month+1
      dtToday=date(date.today().year,m,date.today().day)    
    else:
      dtToday=date.today()

    #dtToday=date(2021,12,1)
    print(type(du),'-',type(fr),'-',type(umMes))
    if pd.isnull(du)==False: 
      intervalo=du+fr*umMes

      if (dtToday<=dp):      
        if (intervalo.month<=dp.month) and (intervalo.year<=dp.year):
          return "Em dia"
        else:
          return "Fora da frequencia"
      else:
        return "Cronograma travado"
    else:
      return 'Não existem preventivas'
a='2021-01-28'
a=datetime.strptime(a, '%Y-%m-%d').date()
p='2022-01-05'
p=datetime.strptime(p, '%Y-%m-%d').date()
#print(type(a),'-',type(p))
#print(a,'-',p)
print(mp_checa_frequencia(a,p,'1 Ano(s)'))

<class 'datetime.date'> - <class 'int'> - <class 'dateutil.relativedelta.relativedelta'>
Em dia


In [ ]:
hoje = date(2020,2,1)
umDia = relativedelta(days=+1)
amanha = hoje + 30*umDia
amanha
hoje.

datetime.date(2020, 2, 1)

In [ ]:

a='2019-03-29'
a=datetime.strptime(a, '%Y-%m-%d').date()
p='2022-01-01'
p=datetime.strptime(p, '%Y-%m-%d').date()
print(type(a),'-',type(p))
if p> a:
  print('sim')
else:
  print('Não')
a.day=p.day
print(a)


NameError: ignored

In [ ]:
#mp[mp.Ativo_Identificador.duplicated()]
pl[pl.Identificador.str.contains('AUTU5-02A')]

,P,Identificador,Data da próxima execução,Data da última execução,Frequência,Nome,Status,OS_Status,OS_Descrição,data_status,MP_Status
404,Atrasado,MP-AUTU5-02A,2021-06-15,2020-12-15,6 Mês(es),Manutenção Preventiva - Autoclave Cisa - Sala ...,Em corretiva,OS-OUT-011298,--,Cronograma travado,Pendente de aprovação: MP-SMC-006407


In [ ]:
def mp_checa_pendencia_aprovacao(ativo):
  '''
  ls_mp=""
  if ativo in mp['SA'].values:
    for ord in mp['Atividade_Identificador'].loc[(mp['Ativo_Identificador']==ativo)].values:
      ls_mp+=","+ord    
    ls_mp=ls_mp[1:]
    return "Pendente de aprovação"+ls_os
  else:
    return "--"
  '''
#-----------------------
  ls_mp=""
  ls_mp_pen=''
  ls_mp_prog=''
  if ativo in mp['Ativo_Identificador'].values:
    pendente=mp[(mp['Ativo_Identificador']==ativo) & (mp['SA']=='Aprovação do Relatório') ]
    programado=mp[(mp['Ativo_Identificador']==ativo) & (mp['SA']!='Aprovação do Relatório') ]
    if ativo in pendente['Ativo_Identificador'].values:
      
      for ord in pendente['Atividade_Identificador'].loc[(pendente['Ativo_Identificador']==ativo)].values: 
        print(type(ord),'--',ord)       
        ls_mp_pen+=","+str(ord)   
      ls_mp="Pendente de aprovação: "+ls_mp_pen[1:]
    if ativo in programado['Ativo_Identificador'].values:

      for ord in programado['Atividade_Identificador'].loc[(programado['Ativo_Identificador']==ativo)].values:   
        print(type(ord),'--',ord)             
        ls_mp_prog+=","+str(ord)    
      ls_mp="Execução: "+ls_mp_prog[1:]
    return ls_mp
  else:
    return "--"
print( mp_checa_pendencia_aprovacao('AUT-00007'))

sim 2021-12-28


In [ ]:
h=date.today()
x=date(2021, 11, 19)
if date.today().day>=18:
  m=date.today().month+1
  y=date(date.today().year,m,date.today().day)
  print('sim',y)
else:
  print(date.today())

In [ ]:

def calD(di,dff):
  if di.month<dff.month:
    return "sim"
def func(): 
  print(calD(dti,dtp))
dtToday=date(2021,8,1)
dt=date(2021,7,14)
dti=plano['Data da última execução'].loc[0].date()
dtp=plano['Data da próxima execução'].loc[0].date()
func()
print(dti,dtp)


sim
2021-06-09 2021-09-09


In [ ]:
for linha in range(0, 1):
  periodo={"Mês(es)":1,"Ano(s)":12,}
  dp=plano['Data da próxima execução'].iloc[linha].date()
  du=plano['Data da última execução'].iloc[linha].date()
  fr=plano['Frequência'].loc[linha].split() 
  dtToday=date(2021,8,1)
  print(fr)
  fr=int(fr[0])*int(periodo[fr[1]])
  #plano.iloc[linha].head()
  umMes=relativedelta(months=+1)
  plano.iloc[0]
  
  ln=str(linha)
  if pd.isnull(du)==False:
    intervalo=du+fr*umMes
    print("Intervalo:"+str(intervalo)+"//Du: "+str(du)+"//DP:"+str(dp))
    if (dtToday<=dp) and (dtToday<=dp):
        print("Entrou1")
        if (intervalo.month<=dp.month) and (intervalo.year<=dp.year):
          print(ln+"Em dia")
          pass
        else:
          print(ln+"Fora da frequencia")
    else:
      print(ln+"Cronograma travado")
  else:
    print("Entrou2")
    if (date.today().month>=dp.month) and (date.today().year>=dp.year):
      print(ln+"Em dia")        
    else:
      print(ln+"Cronograma travado")

['3', 'Mês(es)']
Intervalo:2021-09-09//Du: 2021-06-09//DP:2021-09-09
Entrou1
0Em dia
